In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install spark

In [ ]:
pip install pyspark

In [ ]:
# import necessary libraries
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.linalg import Vectors
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# create a SparkSession
spark = SparkSession.builder.appName("SVM Example").getOrCreate()

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

schema = StructType([
    StructField("age", IntegerType(), True),
    StructField("workclass", StringType(), True),
    StructField("fnlwgt", IntegerType(), True),
    StructField("education", StringType(), True),
    StructField("education-num", IntegerType(), True),
    StructField("marital-status", StringType(), True),
    StructField("occupation", StringType(), True),
    StructField("relationship", StringType(), True),
    StructField("race", StringType(), True),
    StructField("sex", StringType(), True),
    StructField("capital-gain", IntegerType(), True),
    StructField("capital-loss", IntegerType(), True),
    StructField("hours-per-week", IntegerType(), True),
    StructField("native-country", StringType(), True),
    StructField("income", StringType(), True)
])

data = spark.read.csv("/content/drive/MyDrive/cv/adult-training.csv", header=True, schema=schema)
data.show()

In [ ]:

# Create a vector assembler to combine feature columns into a single vector column
assembler = VectorAssembler(inputCols=["age", "fnlwgt", "education-num", "capital-gain", "capital-loss", "hours-per-week"],
                            outputCol="features")

# Apply the vector assembler to the data
data = assembler.transform(data)

# Index the categorical columns using StringIndexer
indexer1 = StringIndexer(inputCol="workclass", outputCol="workclass_index")
indexer2 = StringIndexer(inputCol="education", outputCol="education_index")
indexer3 = StringIndexer(inputCol="marital-status", outputCol="marital_status_index")
indexer4 = StringIndexer(inputCol="occupation", outputCol="occupation_index")
indexer5 = StringIndexer(inputCol="relationship", outputCol="relationship_index")
indexer6 = StringIndexer(inputCol="race", outputCol="race_index")
indexer7 = StringIndexer(inputCol="sex", outputCol="sex_index")
indexer8 = StringIndexer(inputCol="native-country", outputCol="native_country_index")
indexer9 = StringIndexer(inputCol="income", outputCol="label")
data = indexer1.fit(data).transform(data)
data = indexer2.fit(data).transform(data)
data = indexer3.fit(data).transform(data)
data = indexer4.fit(data).transform(data)
data = indexer5.fit(data).transform(data)
data = indexer6.fit(data).transform(data)
data = indexer7.fit(data).transform(data)
data = indexer8.fit(data).transform(data)
data = indexer9.fit(data).transform(data)



In [ ]:

# split the data into training and testing sets
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

In [ ]:
# train an SVM
svm = LinearSVC(maxIter=10, regParam=0.1)
svm_model = svm.fit(train_data)

# make predictions on the test data
predictions = svm_model.transform(test_data)

# evaluate the model using a binary classification evaluator
evaluator = BinaryClassificationEvaluator()
auc = evaluator.evaluate(predictions)
accuracy = evaluator.evaluate(predictions)
print("Confusion Matrix:")
predictions.groupBy("label", "prediction").count().show()
print("Accuracy:", accuracy)
print("Area under ROC curve = {:.2f}".format(auc))